# **Data Preprocessing** 

## **Wunderground Data**

In [3]:
import time
from datetime import datetime, timedelta
import os

import boto3
import pandas as pd
from bs4 import BeautifulSoup as BS
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service

In [4]:
# Set up Firefox options for a headless browser
options = Options()
options.add_argument('--headless')
service = Service(executable_path='C:\Program Files\Geckodriver\geckodriver.exe')

In [5]:
# Function to fetch weather data
def fetch_weather_data(city, formatted_date, start_time, max_retries=10):
    url = f'https://www.wunderground.com/hourly/my/{city}/date/{formatted_date}'
    
    for attempt in range(max_retries):
        try:
            driver = webdriver.Firefox(options=options, service=service)
            driver.get(url)
            time.sleep(30)

            response = BS(driver.page_source, 'html.parser')
            table = response.find('table')
            list_condition = [data.get_text() for i in table.findAll('tr')
                              if (data := i.find('span', {'class': 'show-for-medium conditions'}))]

            driver.quit()

            df = pd.DataFrame()
            df['time'] = pd.date_range(start_time, periods=24, freq=pd.DateOffset(hours=1))
            df['Condition'] = list_condition
            df = df.replace({
                'Fair': 0, 
                'Mostly Clear': 0, 
                'Clear': 0, 
                'Sunny': 0, 
                'Mostly Sunny': 0,
                'Partly Cloudy': 1, 
                'Cloudy': 1, 
                'Mostly Cloudy': 1, 
                'Haze': 2, 
                'Mist': 3,
                'Fog': 4, 
                'Foggy': 4, 
                'Showers in the Vicinity': 5, 
                'Rain': 6, 
                'Few Showers': 7,
                'Showers': 7, 
                'Light Rain': 8, 
                'Light Rain Shower': 9, 
                'Light Rain with Thunder': 10, 
                'Thundershowers': 10,
                'Heavy Rain': 11, 
                'Heavy Rain Shower': 12, 
                'Thunder': 13, 
                'Thunderstorms': 14,
                'Heavy Thunderstorm': 15, 
                'Isolated Thunderstorms': 14, 
                'Scattered Thunderstorms': 14
            })
            df = df.set_index('time')
            df = df.resample('30min').ffill()

            # Directory creation
            directory = f'E:/0. VPP/0. VPP Migration Code/M5 dataset/{city.capitalize()}'
            os.makedirs(directory, exist_ok=True)  # Create directory if it doesn't exist

            # Save to CSV
            csv_path = os.path.join(directory, f'{formatted_date}.csv')
            df.to_csv(csv_path, header=False)
            
            print(f"Data for {city.capitalize()} saved to {csv_path}")
            return df

        except Exception as e:
            print(f"Attempt {attempt + 1} failed for {city}: {e}")
            time.sleep(5)  # Optional: wait before retrying
        finally:
            try:
                driver.quit()
            except:
                pass
    print(f"Failed to fetch")

In [6]:
# Main function
def main():
    start_date = datetime.now()
    current_date = start_date + timedelta(days=1)
    start_time = start_date.replace(hour=16, minute=0, second=0, microsecond=0)
    formatted_date = current_date.strftime("%Y-%m-%d")

    cities = ['kerteh', 'kuantan', 'permatang-pauh', 'malacca', 'segamat']
    for city in cities:
        fetch_weather_data(city, formatted_date, start_time)
    print('Ended')

if __name__ == "__main__":
    main()

Data for Kerteh saved to E:/0. VPP/0. VPP Migration Code/M5 dataset/Kerteh\2025-01-14.csv
Data for Kuantan saved to E:/0. VPP/0. VPP Migration Code/M5 dataset/Kuantan\2025-01-14.csv
Data for Permatang-pauh saved to E:/0. VPP/0. VPP Migration Code/M5 dataset/Permatang-pauh\2025-01-14.csv
Data for Malacca saved to E:/0. VPP/0. VPP Migration Code/M5 dataset/Malacca\2025-01-14.csv
Data for Segamat saved to E:/0. VPP/0. VPP Migration Code/M5 dataset/Segamat\2025-01-14.csv
Ended


## **SOLAR GENERATION**

### **Data Extraction API**

### **Data Cleaning**

In [1]:
import pandas as pd 
import numpy as np

In [2]:
# 
def solarData (solaraData):
    df = pd.Series(solaraData)
    df = df.replace(0.0, np.nan)
    df = df.interpolate(method='linear')
    df = df.fillna(method='bfill')
    df = df.replace(np.nan, 0.0)
    return df.values.tolist()

In [ ]:
# Function to convert power values to kW
def convert_to_kw(value):
    value = str(value).strip()  # Convert to string and strip whitespace
    if 'MW' in value:
        return float(value.replace('MW', '').strip()) * 1000
    elif 'kW' in value:
        return float(value.replace('kW', '').strip())
    elif 'mW' in value:
        return float(value.replace('mW', '').strip()) / 1_000_000
    elif 'W' in value:
        return float(value.replace('W', '').strip()) / 1000
    else:
        return float(value)  # If the value is already in kW

In [13]:
data = pd.read_csv("E:/0. VPP/0. VPP Migration Code/M5 dataset/AC_102023_062024_PP_30m.csv", dayfirst=True, low_memory=False, parse_dates=['DateTime'])


In [14]:
data

DateTime Average Output Power (W)
0    2023-10-01 07:30:00                  75.7 kW
1    2023-10-01 08:00:00                      NaN
2    2023-10-01 08:30:00                      NaN
3    2023-10-01 09:00:00                      NaN
4    2023-10-01 09:30:00                      NaN
...                  ...                      ...
6487 2024-06-30 17:00:00                   396 kW
6488 2024-06-30 17:30:00                   275 kW
6489 2024-06-30 18:00:00                   306 kW
6490 2024-06-30 18:30:00                  87.6 kW
6491 2024-06-30 19:00:00                  14.9 kW

[6492 rows x 2 columns]

In [ ]:
# Apply unit conversion to the 'Average Output Power (W)' column
if 'Average Output Power (W)' in data.columns:
    data['Average Output Power (Watt)'] = data['Average Output Power (W)'].apply(convert_to_kw)
    

In [7]:
# List of columns to repair
columns_to_process = ['Average Output Power (W)']  # Replace with your actual column names

# Apply the solarData function to each column and save the repaired data back to the DataFrame
for col in columns_to_process:
    if col in df.columns:  # Check if the column exists
        df[f'Repaired_{col}'] = solarData(df[col])

In [8]:
df

DateTime Average Output Power (W)  \
0    2023-10-01 07:30:00                  75.7 kW   
1    2023-10-01 08:00:00                      NaN   
2    2023-10-01 08:30:00                      NaN   
3    2023-10-01 09:00:00                      NaN   
4    2023-10-01 09:30:00                      NaN   
...                  ...                      ...   
6487 2024-06-30 17:00:00                   396 kW   
6488 2024-06-30 17:30:00                   275 kW   
6489 2024-06-30 18:00:00                   306 kW   
6490 2024-06-30 18:30:00                  87.6 kW   
6491 2024-06-30 19:00:00                  14.9 kW   

     Repaired_Average Output Power (W)  
0                              75.7 kW  
1                              2.30 MW  
2                              2.30 MW  
3                              2.30 MW  
4                              2.30 MW  
...                                ...  
6487                            396 kW  
6488                            275 kW  
6489                            306 kW  
6490                           87.6 kW  
6491                           14.9 kW  

[6492 rows x 3 columns]

Data Preprocessing 

1. Solar
- Data Extraction 
- Rename, delete duplicate 
- Save raw data into database
- Data understanding (for trainning)
- monthly avrerage
- interpolation 


2. Meter 
3. Load 
4. Battery

